# YOLOv8 Training - Banking Detection Model

Train a custom object detection model on your exported dataset.

**Instructions:**
1. Click Runtime > Change runtime type > Select GPU (T4 is free)
2. Upload your banking dataset ZIP file when prompted
3. Run all cells in order
4. Download trained model when complete

## Step 1: Install Dependencies

In [ ]:
!pip install ultralytics -q

import torch
from ultralytics import YOLO
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
print(f"Ultralytics installed successfully!")

## Step 2: Upload Your Dataset ZIP

Click the upload button and select your banking_dataset ZIP file.

In [ ]:
from google.colab import files
import zipfile
from pathlib import Path

print("Click 'Choose Files' and upload your banking_dataset ZIP")
uploaded = files.upload()

zip_filename = list(uploaded.keys())[0]
print(f"\nUploaded: {zip_filename}")
print(f"Size: {len(uploaded[zip_filename]) / 1024 / 1024:.1f} MB")

## Step 3: Prepare Dataset Structure

In [ ]:
import random
import shutil

print("Preparing dataset...")

# Create directories
base_path = Path('dataset')
dirs = {
    'train_images': base_path / 'images' / 'train',
    'train_labels': base_path / 'labels' / 'train',
    'val_images': base_path / 'images' / 'val',
    'val_labels': base_path / 'labels' / 'val',
    'test_images': base_path / 'images' / 'test',
    'test_labels': base_path / 'labels' / 'test',
}

for dir_path in dirs.values():
    dir_path.mkdir(parents=True, exist_ok=True)

print("Created directory structure")

# Extract ZIP
temp_extract = Path('temp_extract')
temp_extract.mkdir(exist_ok=True)

print(f"Extracting {zip_filename}...")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(temp_extract)

images_dir = temp_extract / 'images'
annotations_dir = temp_extract / 'annotations'

# Get all image files
image_files = list(images_dir.glob('*.jpg'))
print(f"Found {len(image_files)} images")

# Shuffle and split (80/15/5)
random.seed(42)
random.shuffle(image_files)

total = len(image_files)
train_split = int(0.8 * total)
val_split = int(0.95 * total)

train_files = image_files[:train_split]
val_files = image_files[train_split:val_split]
test_files = image_files[val_split:]

print(f"Split: {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")

# Copy files
def copy_split(files, img_dir, label_dir):
    for img_file in files:
        shutil.copy2(img_file, img_dir / img_file.name)
        label_file = annotations_dir / img_file.with_suffix('.txt').name
        if label_file.exists():
            shutil.copy2(label_file, label_dir / label_file.name)

print("Copying files...")
copy_split(train_files, dirs['train_images'], dirs['train_labels'])
copy_split(val_files, dirs['val_images'], dirs['val_labels'])
copy_split(test_files, dirs['test_images'], dirs['test_labels'])

# Read class names
classes_file = temp_extract / 'classes.txt'
if classes_file.exists():
    with open(classes_file, 'r') as f:
        class_names = [line.strip() for line in f.readlines()]
else:
    class_names = ['person', 'car', 'truck', 'handbag', 'backpack', 'bottle', 'cell phone']

print(f"Classes: {', '.join(class_names)}")

# Create dataset.yaml
yaml_content = f"""path: /content/dataset
train: images/train
val: images/val
test: images/test

names:
"""

for idx, class_name in enumerate(class_names):
    yaml_content += f"  {idx}: {class_name}\n"

yaml_path = base_path / 'dataset.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"Created {yaml_path}")

# Clean up
shutil.rmtree(temp_extract)
os.remove(zip_filename)

print("\nDataset preparation complete!")

## Step 4: Train YOLOv8 Model

**Model sizes:**
- yolov8n - Nano (fastest)
- yolov8s - Small (balanced, recommended)
- yolov8m - Medium
- yolov8l - Large

Training takes 15-30 minutes with free GPU.

In [ ]:
# Training configuration
MODEL_SIZE = 's'
EPOCHS = 100
BATCH_SIZE = 16
IMG_SIZE = 640

print("Training Configuration:")
print(f"  Model: YOLOv8{MODEL_SIZE.upper()}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Image size: {IMG_SIZE}x{IMG_SIZE}")
print(f"  Device: GPU ({torch.cuda.get_device_name(0)})")
print(f"\nEstimated time: ~{EPOCHS * 0.2:.0f} minutes")
print("\nStarting training...\n")

In [ ]:
# Load pretrained model
model = YOLO(f'yolov8{MODEL_SIZE}.pt')

# Train the model
results = model.train(
    data='dataset/dataset.yaml',
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=0,
    project='runs/train',
    name='banking_detection',
    pretrained=True,
    optimizer='auto',
    verbose=True,
    seed=42,
    amp=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    fliplr=0.5,
)

print("\nTraining completed!")

## Step 5: Validate Model and View Results

In [ ]:
# Validate on test set
print("Running validation...")
metrics = model.val()

print(f"\nResults:")
print(f"  mAP50: {metrics.box.map50:.3f}")
print(f"  mAP50-95: {metrics.box.map:.3f}")
print(f"  Precision: {metrics.box.mp:.3f}")
print(f"  Recall: {metrics.box.mr:.3f}")

# Display training results
from IPython.display import Image, display

print("\nTraining Results:")

results_dir = Path('runs/train/banking_detection')

if (results_dir / 'confusion_matrix.png').exists():
    print("\nConfusion Matrix:")
    display(Image(filename=str(results_dir / 'confusion_matrix.png')))

if (results_dir / 'results.png').exists():
    print("\nTraining Curves:")
    display(Image(filename=str(results_dir / 'results.png')))

if (results_dir / 'val_batch0_pred.jpg').exists():
    print("\nSample Predictions:")
    display(Image(filename=str(results_dir / 'val_batch0_pred.jpg')))

## Step 6: Test on Sample Image

In [ ]:
# Load best model
best_model = YOLO('runs/train/banking_detection/weights/best.pt')

# Get a random test image
test_images = list(Path('dataset/images/test').glob('*.jpg'))
if test_images:
    test_img = random.choice(test_images)
    
    print(f"Testing on: {test_img.name}")
    
    # Run inference
    results = best_model(test_img)
    
    # Display results
    for result in results:
        result_img = result.plot()
        import cv2
        from matplotlib import pyplot as plt
        
        plt.figure(figsize=(12, 8))
        plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Model Prediction')
        plt.show()
        
        # Print detections
        boxes = result.boxes
        if len(boxes) > 0:
            print(f"\nFound {len(boxes)} objects:")
            for box in boxes:
                cls = int(box.cls[0])
                conf = float(box.conf[0])
                print(f"  - {result.names[cls]}: {conf:.2%}")
        else:
            print("\nNo objects detected in this image")

## Step 7: Download Trained Model

In [ ]:
# Download best model
model_path = 'runs/train/banking_detection/weights/best.pt'
print(f"Downloading: {model_path}")
files.download(model_path)

print("\nDownload complete!")
print("Use this model file (best.pt) in your Video Analytics app")
print("to replace COCO-SSD with your custom trained model!")